<a href="https://colab.research.google.com/github/eunjiWon/SoftwareDefectPredictionMetricUsingDeepLearning/blob/master/Metric_for_Defect_Prediction_Based_on_LSTM_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Test Set Cross-Entropy Using LSTM With Pytorch
I refer this site: 
  [https://machinetalk.org/2019/02/08/text-generation-with-pytorch/](https://machinetalk.org/2019/02/08/text-generation-with-pytorch/)
     

### training_nutch_AllCommitAddLines.txt as a train set and DP_nutch_CommitFiles.txt as a test set

In [22]:
from itertools import chain
token_list = tokenize_data('/content/DP_nutch_CommitFiles.txt')
print(token_list) # 각 라인별로 하나의 리스트로 묶인다. 전체를 하나의 리스트로 만들어야 함.

def tokenize_data(filename):
  with open(filename, "r") as file:
    all_lines = file.readlines()
  list_split_WS = [] 
  list_split_dot = []
  for line in all_lines:
    list_split_WS.append(line.split())
  list_split_WS = list(chain.from_iterable(list_split_WS))
  for token in list_split_WS:
    list_split_dot.append(token.split('.'))
  list_split_dot = list(chain.from_iterable(list_split_dot))
  return list_split_dot




['import', 'org', 'apache', 'hadoop', 'fs', 'FileStatus;', 'private', 'long', 'lastModified', '=', '0;', 'FileStatus', 'stat', '=', 'crawlDbPath', 'getFileSystem(config)', 'getFileStatus(crawlDbPath);', 'long', 'lastModified', '=', 'stat', 'getModificationTime();', 'synchronized', '(this)', '{', 'if', '(readers', '!=', 'null)', '{', 'if', '(this', 'lastModified', '==', 'lastModified)', '{', '//', 'CrawlDB', 'not', 'modified,', 're-use', 'readers', 'return;', '}', 'else', '{', '//', 'CrawlDB', 'modified,', 'close', 'and', 're-open', 'readers', 'closeReaders();', '}', '}', 'this', 'lastModified', '=', 'lastModified;', 'readers', '=', 'MapFileOutputFormat', 'getReaders(crawlDbPath,', 'config);', '}', 'import', 'org', 'apache', 'hadoop', 'fs', 'FileStatus;', 'private', 'long', 'lastModified', '=', '0;', 'public', 'void', 'openReaders()', 'throws', 'IOException', '{', 'Path', 'linkDbPath', '=', 'new', 'Path(directory,', 'LinkDb', 'CURRENT_NAME);', 'FileStatus', 'stat', '=', 'linkDbPath', 'g

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace
from itertools import chain

flags = Namespace(
    train_file='/content/training_nutch_AllCommitAddLines.txt', # training_nutch_AllCommitAddLines.txt
    test_file='/content/DP_nutch_CommitFiles.txt', # DP_nutch_CommitFiles.txt
    seq_size=32, # evolve over t-time steps
    batch_size=16,
    embedding_size=64,
    lstm_size=64, # LSTM hidden size
    gradients_norm=5, # norm to clip gradients
)

def tokenize_data(filename):
  with open(filename, "r") as file:
    all_lines = file.readlines()
  list_split_WS = [] 
  list_split_dot = []
  for line in all_lines:
    list_split_WS.append(line.split())
  list_split_WS = list(chain.from_iterable(list_split_WS))
  for token in list_split_WS:
    list_split_dot.append(token.split('.'))
  list_split_dot = list(chain.from_iterable(list_split_dot))
  return list_split_dot

def get_data_from_file(file, batch_size, seq_size):
  text = tokenize_data(file)
  # Create two dictionaries, one to convert words into integers indices, 
  # and the other one to convert integer indices back to word tokens
  word_counts = Counter(text)
  sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
  int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
  vocab_to_int = {w: k for k, w in int_to_vocab.items()}
  n_vocab = len(int_to_vocab)
  print('Vocabulary size', n_vocab)
  
  # Covert word tokens into integer indices. 
  # These will be the input to the network
  # We will train a mini-batch each iteration 
  # so we split the data into batches evenly. 
  # Chopping out the last uneven batch
  int_text = [vocab_to_int[w] for w in text]
  num_batches = int(len(int_text) / (seq_size * batch_size))
  in_text = int_text[:num_batches * batch_size * seq_size]
  
  # In next generation problem, 
  # the target of each input word will be its consecutive wold,
  # so we just shift the whole input data to the left by one step
  out_text = np.zeros_like(in_text)
  out_text[:-1] = in_text[1:] # in_text의 두번째 부터 out_text의 처음으로 복사
  out_text[-1] = in_text[0] # in_text의 처음을 out_text의 마지막으로 복사
  in_text = np.reshape(in_text, (batch_size, -1))
  out_text = np.reshape(out_text, (batch_size, -1))
  # print(in_text[:10][:10]) # top and left of matrix
  # print(out_text[:10][:10]) # top and left of matrix
  return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text
  
def get_batches(in_text, out_text, batch_size, seq_size):
  num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
  for i in range(0, num_batches * seq_size, seq_size):
      yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

class RNNModule(nn.Module):
  def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
    super(RNNModule, self).__init__()
    self.seq_size = seq_size
    self.lstm_size = lstm_size
    self.embedding = nn.Embedding(n_vocab, embedding_size)
    self.lstm = nn.LSTM(embedding_size, lstm_size, batch_first=True)
    self.dense = nn.Linear(lstm_size, n_vocab)

  # Take an input sequence and the previous states (hidden states) and produce the output together with states of the currents timestamp
  def forward(self, x, prev_state):
    embed = self.embedding(x)
    output, state = self.lstm(embed, prev_state)
    logits = self.dense(output)
    return logits, state # why return state variable?
  
  # Define one more method to help us set states to zero because we need to reset states at the beginning of every epoch.
  def zero_state(self, batch_size):
    return (torch.zeros(1, batch_size, self.lstm_size), # hidden state (the short-term memory)
            torch.zeros(1, batch_size, self.lstm_size)) # cell state (the long-term memory)

def get_loss_and_train_op(net, lr=0.001):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  return criterion, optimizer
  # gradient clipping doesn't apply here!

def train(in_text, out_text, flags, net, device, criterion, optimizer, e):
  batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
  state_h, state_c = net.zero_state(flags.batch_size)
  # Transfer data to GPU
  state_h = state_h.to(device)
  state_c = state_c.to(device)

  iteration = 0
  for x, y in batches: # x is in_text and y is out_text
    iteration += 1
    # Tell it we are in training mode
    net.train()
    # Reset all gradients
    optimizer.zero_grad()
    # Transfer data to GPU
    x = torch.tensor(x).to(device)
    y = torch.tensor(y).to(device)
    logits, (state_h, state_c) = net(x, (state_h, state_c))
    loss = criterion(logits.transpose(1, 2), y) # why we transpose the logits?
    # Avoid autograd which is given by Pytorch to keep track of the tensor's flow to perform back-propagation.
    state_h = state_h.detach()
    state_c = state_c.detach()
    loss_value = loss.item() # this loss is cross-entropy which is thing I want!!! 
    # Perform back-propagation
    loss.backward()
    # Gradient clipping
    _ = torch.nn.utils.clip_grad_norm_(net.parameters(), flags.gradients_norm)
    # Update the network's parameters
    optimizer.step() # the number of parameters update is batch-size * epoch
  # Print the loss value and have the model generate some text for us during training
  print('Epoch: {}/{}'.format(e, 200), 'Loss (C.E): {}'.format(loss_value)) # here, we just print the size of epoch 

def test(test_in_text, test_out_text, net, device, criterion):
  net.eval() # Tell it we are in evaluation mode
  # 학습된 parameters을 이용하고 hidden and cell states는 초기화 시켜야함. 
  state_h, state_c = net.zero_state(1) #
  state_h = state_h.to(device) #
  state_c = state_c.to(device) #
  # Transfer data to GPU
  x = torch.tensor(test_in_text).to(device)
  y = torch.tensor(test_out_text).to(device)
  logits, (state_h, state_c) = net(x, (state_h, state_c))
  loss = criterion(logits.transpose(1, 2), y) # why we transpose the logits?
  loss_value = loss.item() # this loss is cross-entropy which is thing I want!!! 
  print("test set loss value (C.E.): ", loss_value)

def main():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
      flags.train_file, flags.batch_size, flags.seq_size)
  test_int_to_vocab, test_vocab_to_int, test_n_vocab, test_in_text, test_out_text = get_data_from_file(
      flags.test_file, 1, flags.seq_size)
  net = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
  net = net.to(device)
  criterion, optimizer = get_loss_and_train_op(net, 0.01)
  
  # Training
  for e in range(50):
    train(in_text, out_text, flags, net, device, criterion, optimizer, e)
  
  # Test
  test(test_in_text, test_out_text, net, device, criterion)
    
if __name__ == '__main__':
  main()




Vocabulary size 65486
Vocabulary size 64
Epoch: 0/200 Loss (C.E): 4.7521891593933105
Epoch: 1/200 Loss (C.E): 3.5346152782440186
Epoch: 2/200 Loss (C.E): 2.9828176498413086
Epoch: 3/200 Loss (C.E): 2.640777826309204
Epoch: 4/200 Loss (C.E): 2.455153703689575
Epoch: 5/200 Loss (C.E): 2.2705812454223633
Epoch: 6/200 Loss (C.E): 2.161515712738037
Epoch: 7/200 Loss (C.E): 2.1456027030944824
Epoch: 8/200 Loss (C.E): 2.0428504943847656
Epoch: 9/200 Loss (C.E): 2.0305047035217285
Epoch: 10/200 Loss (C.E): 2.05074143409729
Epoch: 11/200 Loss (C.E): 2.0234522819519043
Epoch: 12/200 Loss (C.E): 2.001739263534546
Epoch: 13/200 Loss (C.E): 2.0418975353240967
Epoch: 14/200 Loss (C.E): 1.9405914545059204
Epoch: 15/200 Loss (C.E): 1.9398503303527832
Epoch: 16/200 Loss (C.E): 1.9209319353103638
Epoch: 17/200 Loss (C.E): 1.8933955430984497
Epoch: 18/200 Loss (C.E): 1.8901100158691406
Epoch: 19/200 Loss (C.E): 1.9184101819992065
Epoch: 20/200 Loss (C.E): 1.804491400718689
Epoch: 21/200 Loss (C.E): 1.818